In [31]:
import pandas as pd

berichtinteracties_met_sentiment = pd.read_csv("Datasets/berichtinteracties_met_sentiment.csv",sep=';')
evenementaanwezigheid = pd.read_csv("Datasets/evenementaanwezigheid.csv",sep=';')
evenementinformatie = pd.read_csv("Datasets/evenementinformatie.csv",sep=';')
gebruikersinformatie = pd.read_csv("Datasets/gebruikersinformatie.csv",sep=';')


berichtinteracties_met_sentiment = berichtinteracties_met_sentiment.drop_duplicates()
evenementaanwezigheid = evenementaanwezigheid.drop_duplicates()
evenementinformatie = evenementinformatie.drop_duplicates()
gebruikersinformatie = gebruikersinformatie.drop_duplicates()


In [38]:
gebruikersinformatie['RegistratieDatum'] = pd.to_datetime(gebruikersinformatie['RegistratieDatum'], format='%d-%m-%Y', errors='coerce')
gebruikersinformatie['LaatsteLogin'] = pd.to_datetime(gebruikersinformatie['LaatsteLogin'], format='%d-%m-%Y', errors='coerce')

true_rows = gebruikersinformatie['RegistratieDatum'] > gebruikersinformatie['LaatsteLogin']

print(true_rows)

0      False
1      False
2      False
3      False
4      False
       ...  
315    False
316    False
317    False
318    False
319    False
Length: 320, dtype: bool


In [39]:
# Filter rows where conditions are met
merged_data = pd.merge(
    evenementaanwezigheid,
    berichtinteracties_met_sentiment,
    on=["EvenementID", "GebruikerID"],
    how="inner"  # Alleen rijen die overeenkomen worden behouden
)


In [67]:
even = evenementaanwezigheid.drop_duplicates()
newdata = even[even["Aanwezigheidsstatus"] == "Afwezig"]
print(newdata)



grouped_data = even.groupby(["EvenementID", "GebruikerID"]).agg(
    Count=("Aanwezigheidsstatus", "size"),  # Aantal rijen per groep
    AanwezigheidsstatusList=("Aanwezigheidsstatus", list)  # Lijst van Aanwezigheidsstatus
).reset_index()

duplicates = grouped_data[grouped_data["Count"] > 1]
is_Afwezig = duplicates[duplicates["AanwezigheidsstatusList"].apply(lambda x: "Afwezig" in x and "Bijgewoond" in x)]

print(is_Afwezig)



     EvenementID GebruikerID Aanwezigheidsstatus
0           E003        G131             Afwezig
1           E016        G127             Afwezig
2           E034        G309             Afwezig
4           E022        G083             Afwezig
6           E026        G060             Afwezig
...          ...         ...                 ...
3977        E037        G093             Afwezig
3986        E004        G141             Afwezig
3992        E003        G114             Afwezig
3996        E017        G114             Afwezig
3999        E010        G148             Afwezig

[1250 rows x 3 columns]
     EvenementID GebruikerID  Count               AanwezigheidsstatusList
24          E001        G107      2                 [Afwezig, Bijgewoond]
38          E001        G163      2                 [Afwezig, Bijgewoond]
113         E002        G142      2                 [Afwezig, Bijgewoond]
141         E002        G246      2                 [Bijgewoond, Afwezig]
150         E002 

In [68]:
evenementaanwezigheid = evenementaanwezigheid[
    ~(
        (evenementaanwezigheid["EvenementID"].isin(is_Afwezig["EvenementID"])) &
        (evenementaanwezigheid["GebruikerID"].isin(is_Afwezig["GebruikerID"])) &
        (evenementaanwezigheid["Aanwezigheidsstatus"] == "Afwezig")
    )
]
print(evenementaanwezigheid)

     EvenementID GebruikerID Aanwezigheidsstatus
0           E003        G131             Afwezig
1           E016        G127             Afwezig
2           E034        G309             Afwezig
3           E020        G297          Bijgewoond
5           E032        G007          Bijgewoond
...          ...         ...                 ...
3993        E024        G116          Bijgewoond
3994        E002        G010       Geregistreerd
3995        E010        G167       Geregistreerd
3997        E039        G033          Bijgewoond
3998        E030        G314          Bijgewoond

[3321 rows x 3 columns]


In [41]:
# newdata = merged_data[merged_data["Aanwezigheidsstatus"] == "Afwezig"]
# print(newdata)



grouped_data = merged_data.groupby(["EvenementID", "GebruikerID"]).agg(
    Count=("Aanwezigheidsstatus", "size"),  # Aantal rijen per groep
    AanwezigheidsstatusList=("Aanwezigheidsstatus", list),  # Lijst van Aanwezigheidsstatus
).reset_index()

duplicates = grouped_data[grouped_data["Count"] > 0]
is_Afwezig = duplicates[duplicates["AanwezigheidsstatusList"].apply(lambda x: len(set(x))==1 and "Afwezig" in x)]

print(is_Afwezig)

    EvenementID GebruikerID  Count AanwezigheidsstatusList
2          E001        G237      1               [Afwezig]
4          E002        G270      1               [Afwezig]
6          E003        G073      1               [Afwezig]
7          E003        G074      1               [Afwezig]
8          E003        G084      1               [Afwezig]
12         E004        G227      1               [Afwezig]
16         E006        G144      1               [Afwezig]
20         E007        G215      1               [Afwezig]
24         E008        G073      1               [Afwezig]
31         E009        G035      2      [Afwezig, Afwezig]
34         E009        G214      1               [Afwezig]
38         E011        G153      1               [Afwezig]
41         E012        G137      1               [Afwezig]
44         E013        G034      1               [Afwezig]
46         E013        G093      1               [Afwezig]
53         E014        G304      1               [Afwezi

In [42]:
result = pd.merge(is_Afwezig, berichtinteracties_met_sentiment, on=["EvenementID", "GebruikerID"], how="inner")

print(result)



   EvenementID GebruikerID  Count AanwezigheidsstatusList BerichtID  \
0         E001        G237      1               [Afwezig]      B022   
1         E002        G270      1               [Afwezig]      B147   
2         E003        G073      1               [Afwezig]      B161   
3         E003        G074      1               [Afwezig]      B407   
4         E003        G084      1               [Afwezig]      B237   
5         E004        G227      1               [Afwezig]      B436   
6         E006        G144      1               [Afwezig]      B077   
7         E007        G215      1               [Afwezig]      B225   
8         E008        G073      1               [Afwezig]      B203   
9         E009        G035      2      [Afwezig, Afwezig]      B153   
10        E009        G035      2      [Afwezig, Afwezig]      B217   
11        E009        G214      1               [Afwezig]      B432   
12        E011        G153      1               [Afwezig]      B274   
13    

In [52]:

berichtinteracties_met_sentiment = berichtinteracties_met_sentiment[
    ~(
        (berichtinteracties_met_sentiment["EvenementID"].isin(result["EvenementID"])) &
        (berichtinteracties_met_sentiment["GebruikerID"].isin(result["GebruikerID"]))
    )
]
print(berichtinteracties_met_sentiment)


    BerichtID BerichtDatum GebruikerID EvenementID BerichtSentiment  \
0        B001    18-5-2023        G309        E012         Neutraal   
1        B002    6-10-2023        G021        E039         Positief   
2        B003     7-7-2023        G111        E001         Positief   
3        B004    5-10-2023        G168        E015         Negatief   
4        B005    22-1-2023        G076        E017         Neutraal   
..        ...          ...         ...         ...              ...   
494      B495     3-3-2023        G210        E025         Positief   
495      B496    16-2-2023        G263        E010         Neutraal   
497      B498   18-12-2023        G005        E004         Positief   
498      B499     5-6-2023        G243        E013         Positief   
499      B500   27-12-2023        G068        E018         Positief   

     OntvangenLikes  OntvangenReacties  
0                 5                  8  
1                 9                  6  
2                 8     

In [37]:

newdata = merged_data[merged_data["Aanwezigheidsstatus"] == "Afwezig"]

print(newdata)

    EvenementID GebruikerID Aanwezigheidsstatus BerichtID BerichtDatum  \
0          E027        G204             Afwezig      B371    19-1-2023   
1          E009        G214             Afwezig      B432    23-2-2023   
2          E018        G151             Afwezig      B288    15-4-2023   
5          E012        G137             Afwezig      B114    24-1-2023   
8          E013        G244             Afwezig      B416     6-5-2023   
10         E006        G144             Afwezig      B077   21-11-2023   
11         E019        G294             Afwezig      B050   14-12-2023   
14         E009        G145             Afwezig      B273     1-8-2023   
18         E022        G155             Afwezig      B171     6-2-2023   
19         E036        G089             Afwezig      B353    22-3-2023   
23         E003        G073             Afwezig      B161    16-8-2023   
24         E030        G154             Afwezig      B012     7-5-2023   
25         E008        G138           

In [69]:
berichtinteracties_met_sentiment
evenementaanwezigheid 
evenementinformatie
gebruikersinformatie

,GebruikerID,RegistratieDatum,LaatsteLogin,Leeftijd,Studierichting,Studiejaar,Lidmaatschapstype
0,G001,2023-11-24,2024-05-02,21.0,Software engineering,Eerstejaars,Regulier
1,G002,2023-02-27,2023-03-28,21.0,Data engineering,Eerstejaars,Premium
2,G003,2023-01-13,2023-02-07,22.0,Security & Cloud,Vierdejaars,Premium
3,G004,2023-05-21,2024-03-15,17.0,Human computer interaction,Eerstejaars,Regulier
4,G005,2023-05-06,2024-01-05,17.0,Human computer interaction,Eerstejaars,Premium
...,...,...,...,...,...,...,...
315,G316,2023-04-22,2023-12-17,17.0,Data Engineering,Tweedejaars,Regulier
316,G317,2023-01-30,2023-04-05,23.0,Data engineering,Vierdejaars,Regulier
317,G318,2023-10-24,2024-07-23,NaN,Data engineering,Derdejaars,Premium
318,G319,2023-10-05,2023-10-18,23.0,Human computer interaction,Derdejaars,Premium
